# Machine Learning using SKlearn
Ultimately, we want to see which variables have the greatest impact on AQI. To do this we must perform a machine learning analysis and create a prediction algorithm. 


In [ ]:
#| label: import-packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

Starting by looking at the head to see all of our columns. Immediately, we can some missing data, which must be addressed before any machine learning algorithms can be conducted.

## Import the Data

In [ ]:
#| label: read-csv
df = pd.read_csv('https://raw.githubusercontent.com/wu-msds-capstones/Air-Quality-Index/main/data/metro_1mil.csv')
#df = pd.read_csv('https://github.com/wu-msds-capstones/Air-Quality-Index/blob/main/data/metro_1mil.csv')

In [ ]:
#| label: check-len
len(df)

Here we can see all the missing data in each column. There are some very large numbers we must address. The entire dataset is only 147039 rows long. Some columnshave more than half the data missing.


In [ ]:
#| label: check-isna
df.isna().sum()

The first steps to addressing the missing data will be to drop the columns that will definitely not be needed. This includes the id columns used for joining within the SQL database, as well as the maximum columns as we already have the mean data. We also rename the columns to make it easier to understand and reference.


In [ ]:
#| label: met-dataframe
met = df[[
    'date',
    'state',
    'county',
    'city',
    'population',
    'density',
    'aqi',
    'category',
    'mean_temperature_fahrenheit',
    'mean_pressure_millibars',
    'mean_humidity_percent_relative_humidity',
    'mean_wind_knots',
    'mean_co_ppm',
    'mean_no2_ppb',
    'mean_ozone_ppm',
    'mean_so2_ppb',
    'mean_pm100_micrograms_per_cubic_meter',
    'mean_pm25_micrograms_per_cubic_meter',
    'mean_lead_micrograms_per_cubic_meter',
    'num_busses',
    'revenue',
    'operating_expense',
    'passenger_trips',
    'operating_hours',
    'passenger_miles',
    'operating_miles'
    ]]

met = met.rename(columns={'mean_temperature_fahrenheit': 'temp', 
                          'mean_pressure_millibars': 'pressure', 
                          'mean_humidity_percent_relative_humidity': 'humidity',
                          'mean_wind_knots': 'wind_speed', 
                          'mean_co_ppm': 'co', 
                          'mean_no2_ppb': 'no2',
                          'mean_ozone_ppm': 'o3', 
                          'mean_so2_ppb': 'so2', 
                          'mean_pm100_micrograms_per_cubic_meter': 'pm100',
                          'mean_pm25_micrograms_per_cubic_meter': 'pm25', 
                          'mean_lead_micrograms_per_cubic_meter': 'lead'})

Now the missing data looks like this, it is a little better, but we can still do so much more.


In [ ]:
#| label: check-NaN
met.isna().sum()

We will group data by week. This will take an average count for the entire week for each variable for each city. This will eliminate any missing data that was not captured everyday, but only some days. For example, the pollutants (PM10 and PM2.5) were captured once every three days. 


In [ ]:
#| label: set-index-ts
met['date'] = pd.to_datetime(met['date'])
met.set_index('date', inplace=True)
#group by week, drop lead for too much missing data
met = met.groupby([pd.Grouper(freq='W'), 'state', 'county', 'city']).agg({
    'population': 'first',
    'density': 'first',
    'aqi': 'mean',
    'temp': 'mean',
    'pressure': 'mean',
    'humidity': 'mean',
    'wind_speed': 'mean',
    'co': 'mean',
    'no2': 'mean',
    'o3': 'mean',
    'so2': 'mean',
    'pm100': 'mean',
    'pm25': 'mean',
    'num_busses': 'mean',
    'revenue': 'mean',
    'operating_expense': 'mean',
    'passenger_trips': 'mean',
    'operating_hours': 'mean',
    'passenger_miles': 'mean',
    'operating_miles': 'mean'
}).reset_index()

We can also see that the city Virginia Beach has almost no data, so it will be easiest to drop that city. 


In [ ]:
#| label: drop-missNaN-col
met = met[met['city'] != 'Virginia Beach']

Next, since we will be measuring and predicting AQI, we will drop all rows missing AQI data.


In [ ]:
#| label: drop-null-aqi
met = met.dropna(subset=['aqi'])

Missing data is better, but there is still work to do.


In [ ]:
#| label: check-NaN-again
met.isna().sum()

The data collected has separate information for the city of New York City. NYC is divided into five borroughs, each within its own county. We will group and average out these values to make NYC have the same amount of datapoints as every other city. This will also address data present in some New York borroughs but not others. 


In [ ]:
#| label: group-nyc-1-city
nyc = met[met['city'] == 'New York']
columns = ['date', 'aqi', 'temp', 'pressure','humidity',
           'wind_speed','co','no2','o3',
           'so2','pm100','pm25', 'num_busses',
           'revenue', 'operating_expense', 
           'passenger_trips', 'operating_hours', 
           'passenger_miles', 'operating_miles']

nyc = nyc[columns].groupby('date').mean().reset_index()

#Add data that was dropped
nyc['state'] = 'New York'
nyc['county'] = 'Multiple'
nyc['city'] = 'New York City'
nyc['population'] = 18908608
nyc['density'] = 11080.3

nyc = nyc[['date', 'state', 'county', 'city', 'population', 
     'density', 'aqi', 'temp', 'pressure','humidity',
     'wind_speed','co','no2','o3', 'so2','pm100',
     'pm25', 'num_busses', 'revenue', 'operating_expense', 
     'passenger_trips', 'operating_hours', 
     'passenger_miles', 'operating_miles']]

print(nyc)

In [ ]:
#| label: replace-nyc-in-met
met = met[met['city'] != 'New York']
met = pd.concat([met, nyc], ignore_index=True)

Additionally, Kansas City is located in two states, and thus in two counties. We will do the same thing to contract and standardize the data.


In [ ]:
#| label: dataframe-Kansas-City
#Do same with Kansas City
kc = met[met['city'] == 'Kansas City']
columns = ['date', 'aqi', 'temp', 'pressure','humidity',
           'wind_speed','co','no2','o3',
           'so2','pm100','pm25', 'num_busses',
           'revenue', 'operating_expense', 
           'passenger_trips', 'operating_hours', 
           'passenger_miles', 'operating_miles']

kc = kc[columns].groupby('date').mean().reset_index()

columns = ['temp', 'pressure','humidity',
           'wind_speed','co','no2','o3',
           'so2','pm100','pm25', 'num_busses',
           'revenue', 'operating_expense', 
           'passenger_trips', 'operating_hours', 
           'passenger_miles', 'operating_miles']
met[columns] = met[columns].fillna(met.groupby('city')[columns].transform('mean'))

kc['state'] = 'Missouri'
kc['county'] = 'Multiple'
kc['city'] = 'Kansas City'
kc['population'] = 1689556
kc['density'] = 620.7

kc = kc[['date', 'state', 'county', 'city', 'population', 
     'density', 'aqi', 'temp', 'pressure','humidity',
     'wind_speed','co','no2','o3', 'so2','pm100',
     'pm25', 'num_busses', 'revenue', 'operating_expense', 
     'passenger_trips', 'operating_hours', 
     'passenger_miles', 'operating_miles']]

met = met[met['city'] != 'Kansas City']
met = pd.concat([met, kc], ignore_index=True)

After doing this, we drop the rest of the rows with missing data, leaving us with a total of 17 cities with metropolitan area populations greater than one million.


In [ ]:
#| label: count-cities
#count cities after dropping na rows, left with 17 cities
ds = met.dropna()
ds.groupby('city').size().reset_index(name='count')

To perform a ML prediction algorithm, the predicted variable (AQI) must be discrete. To achieve this, we bin AQI data into discrete groups. Initially, we thought to bin them based on the existing AQI categories, but found that most of the data is grouped in the sub 100 range. Therefore, we expanded the bins, focusing the prediction on outcomes in the double digits. The bins chosen are shown below:


In [ ]:
#| label: AQI-bin-ML
#bin AQI data for ML
bins = [0, 30, 40, 50, 60, 70, 80, 90, 100, 150, 500]

labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
ds.loc[:,'aqi_discrete'] = pd.cut(ds['aqi'], bins=bins, labels=labels, right=True)

ds.head()

In [ ]:
#| label: label-bin
labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
ds.loc[:,'aqi_discrete'] = pd.cut(ds['aqi'], bins=bins, labels=labels, right=True)

ds.head()

In [ ]:
#| label: convert-year-month-datetime
ds.loc[:,'year'] = pd.to_datetime(ds['date']).dt.year
ds.loc[:,'month'] = pd.to_datetime(ds['date']).dt.month

To set up the ML prediction, we must specify the variable being predicted (aqi_discrete, the aqi separated into bins), and the independent prediction variables. We will use a feature selector to select the most accurate predictors to try to optimize the model. Therefore, we initially choose every other variable to be our independent variables.


In [ ]:
#| label: aqi-X-y
#dependent variable aqi, all others (except aqi category) as independent variables
y = ds['aqi_discrete']
X = ds[['city',
        'population',
        'density',
        'temp',
        'pressure',
        'humidity',
        'wind_speed',
        'co',
        'no2',
        'o3',
        'so2',
        'pm100',
        'pm25',
        'num_busses', 
        'revenue', 
        'operating_expense', 
        'passenger_trips', 
        'operating_hours', 
        'passenger_miles', 
        'operating_miles',
        'year',
        'month'
        ]]

We must split the data into training and testing datasets. We also define an encoder to transform the discrete predictors into many binary variables for each category.


In [ ]:
#| label: split-train
#Split into train and test datasets, define model and encoder
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
encoder = OneHotEncoder(sparse_output=False, min_frequency=5, handle_unknown='infrequent_if_exist')

A transformer is used to quickly transform variables into the necessary form for modeling.


In [ ]:
#| label: def-transformer
#define transformer with encoder and standardscaler

transformer = ColumnTransformer(
        [
            ('categories', encoder, ['city','year','month']),
            ('scaled_air_quality', StandardScaler(), [
                'population',
                'density',
                'temp',
                'pressure',
                'humidity',
                'wind_speed',
                'co',
                'no2',
                'o3',
                'so2',
                'pm100',
                'pm25',
                'num_busses', 
                'revenue', 
                'operating_expense', 
                'passenger_trips', 
                'operating_hours', 
                'passenger_miles', 
                'operating_miles'
                ]
            )
        ],
        remainder='drop', verbose_feature_names_out=False)
#fit transformer
transformer.fit(X_train, y_train)

Feature selection is done on the data. From this we are given the following variables: City, Temperature, Humidity, Carbon Monoxide, Nitrogen Dioxide, Ozone, PM10, and PM2.5.


In [ ]:
#| label: feature-select
#feature selection
feature_selector = SelectKBest(k=10)
X_train_trans = transformer.transform(X_train)
X_train_trans_df = pd.DataFrame(
    X_train_trans, 
    columns = transformer.get_feature_names_out(),
    index = X_train.index)
feature_selector.fit(X_train_trans_df, y_train)
feature_selector.get_support()
feature_selector.scores_[feature_selector.get_support()]
X_train_trans_df.columns[feature_selector.get_support()]

Using these selected features, we will run our models.


In [ ]:
#| label: use-features
#Use selected features
y = ds['aqi_discrete']
X = ds[[
    'city', 'temp', 'humidity', 'co', 'no2', 'o3', 'pm100', 'pm25'
    ]]
#Create train/test data with selected features
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train_enc = pd.DataFrame(encoder.fit_transform(X_train[['city']]),
                           columns = encoder.get_feature_names_out(), index = X_train.index)
X_train_trans = X_train[[
    'temp', 'humidity', 'co', 'no2','o3', 'pm100', 'pm25'
    ]].merge(X_train_enc, left_index = True, right_index = True)
X_test_enc = pd.DataFrame(encoder.fit_transform(X_test[['city']]),
                           columns = encoder.get_feature_names_out(), index = X_test.index)
X_test_trans = X_test[[
    'temp', 'humidity', 'co', 'no2','o3', 'pm100', 'pm25'
    ]].merge(X_test_enc, left_index = True, right_index = True)

We will test various models with default parameters to see which is the most accurate at predicting AQI from this dataset. The five models selected are: K nearest neighbors, Tree model, Random Forest model, Logistic Regression, and Naive Bayes.Running the models shows us that the Random Forest model is the most accurate.


In [ ]:
#| label: model-select
#model selection
model1 = KNeighborsClassifier(n_neighbors=5)
model2 = tree.DecisionTreeClassifier()
model3 = RandomForestClassifier(n_estimators=10, random_state=12)
model4 = LogisticRegression()
model5 = GaussianNB()

for model, label in zip([model1, model2, model3, model4, model5], ['KNN', 'Tree', 'Random Forest', 'Logistic', 'naive Bayes']):
    model.fit(X_train_trans, y_train)
    y_pred = model.predict(X_test_trans)
    print("Model: ", label)
    print("Cohen Kappa Score: ", cohen_kappa_score(y_test, y_pred))
    print("Accuracy: ", sum(y_test == y_pred)/len(y_test))
    print("\n")

Running a basic Random Forest model gives us the following cohen kappa score and accuracy. This will serve as the baseline to be compared to.


In [ ]:
#| label: def-best-model
#Create model from most accurate, RandomForest
model = RandomForestClassifier(n_estimators=10)
model.fit(X_train_trans, y_train)

#predict on test data
y_pred = model.predict(X_test_trans)

#Print results
print("cohen kappa score: ", cohen_kappa_score(y_test, y_pred))
print("accuracy: ", sum(y_test == y_pred)/len(y_test))
ConfusionMatrixDisplay(
    confusion_matrix = confusion_matrix(
        y_test, y_pred, 
        labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
        ), 
        display_labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
        ).plot(xticks_rotation='vertical')

We will further simplify this with a transformer to be used in a pipeline.


In [ ]:
#| label: improve-transformer
#redefined transformer
transformer = ColumnTransformer(
        [
            ('categories', encoder, ['city']),
            ('scaled_air_quality', StandardScaler(), [
                'temp',
                'humidity',
                'co',
                'no2',
                'o3',
                'pm100',
                'pm25'
                ]
            )
        ],
        remainder='drop', verbose_feature_names_out=False)
#Pipeline for simplification

classification_pipeline = Pipeline([('aqi_transformer', transformer),
                                    ('RF_model', RandomForestClassifier())
                                    ])
classification_pipeline.fit(X_train, y_train)

Next we will optimize hyperparameters to make the model more accurate. 


In [ ]:
#| label: hyper-parameters-optim
#hyperparameter optimization
classification_pipeline.get_params()

The following parameters are selected, each with a range of possible values. A large 100 iterations are run to find the highest score.


In [ ]:
#| label: def-parameters
parameters = {
    'aqi_transformer__categories__max_categories': randint(3,30),
    'aqi_transformer__categories__min_frequency': randint(2,20),
    'RF_model__max_depth': randint(3, 30),
    'RF_model__max_features': [None, 'sqrt', 'log2'],
    'RF_model__min_samples_leaf': randint(2, 10),
    'RF_model__min_samples_split': randint(2, 10),
    'RF_model__n_estimators': randint(50, 200),
    'RF_model__min_samples_split': randint(2, 10),
    'RF_model__bootstrap': [True, False]
}
n_iter_search = 10 # be 100
random_search = RandomizedSearchCV(classification_pipeline, param_distributions=parameters,
                                   n_iter=n_iter_search, n_jobs=-1, cv=5)
random_search.fit(X_train, y_train)
print(random_search.best_score_)

We can see which hyperparameters are selected.


In [ ]:
#| label: randsearch-get-params
random_search.best_estimator_.get_params()

Using these hyperparameters, we reach an accuracy of about 63%.


In [ ]:
#| label: predict-X_test
y_pred=random_search.predict(X_test)
print("cohen kappa score: ", cohen_kappa_score(y_test, y_pred))
print("accuracy: ", sum(y_pred == y_test)/len(y_test))
ConfusionMatrixDisplay(
    confusion_matrix = confusion_matrix(
        y_test, y_pred, 
        labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
        ), 
        display_labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
        ).plot(xticks_rotation='vertical')

We can use this model for the prediction of AQI, exploring the features that predict it, and use that to generate conclusions for what particles to reduce or systems to increase.